In [20]:
import torch

# Assuming you have already loaded your text data into 'text'
with open('wiz_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Create the vocabulary
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_len)




['\n', ' ', '!', '&', '(', ')', ',', '-', '.', '0', '1', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”']
72


In [21]:
# Create mappings from character to integer and vice versa
string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}

# Encoding and decoding functions
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# Encode the entire text data
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

# Define hyperparameters
block_size = 8  # Length of sequences for inputs
batch_size = 4  # Number of sequences in a batch

# Device configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Split data into training and validation sets
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

# Function to get a batch of data
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# Get a batch of training data
x, y = get_batch('train')

print('Inputs:')
print(x)
print('Targets:')
print(y)

tensor([ 0, 34, 48, 45,  1, 37, 55, 54, 44, 45, 58, 46, 61, 52,  1, 37, 49, 66,
        41, 58, 44,  1, 55, 46,  1, 29, 66,  0,  0, 42, 65,  1, 26,  8,  1, 20,
        58, 41, 54, 51,  1, 16, 41, 61, 53,  0,  0,  0, 34, 48, 49, 59,  1, 42,
        55, 55, 51,  1, 49, 59,  1, 44, 45, 44, 49, 43, 41, 60, 45, 44,  1, 60,
        55,  1, 53, 65,  1, 47, 55, 55, 44,  1, 46, 58, 49, 45, 54, 44,  1,  3,
         1, 43, 55, 53, 58, 41, 44, 45,  0, 27])
Inputs:
tensor([[55,  1, 41, 60,  0, 55, 54, 43],
        [ 6,  0, 63, 48, 55,  1, 48, 41],
        [ 0,  0, 33, 55,  1, 60, 48, 45],
        [56, 55, 51, 45,  1, 60, 55,  0]], device='cuda:0')
Targets:
tensor([[ 1, 41, 60,  0, 55, 54, 43, 45],
        [ 0, 63, 48, 55,  1, 48, 41, 44],
        [ 0, 33, 55,  1, 60, 48, 45,  1],
        [55, 51, 45,  1, 60, 55,  0, 48]], device='cuda:0')


In [22]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()  # Set the model to evaluation mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)  # Initialize an array to hold the loss values
        for k in range(eval_iters):
            X, Y = get_batch(split)  # Get a batch of input and target data
            logits, loss = model(X, Y)  # Forward pass through the model
            losses[k] = loss.item()  # Store the loss as a float
        out[split] = losses.mean()  # Average the loss over eval_iters
    model.train()  # Set the model back to training mode
    return out


In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the BigramModel, inheriting from nn.Module
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # The embedding table: Each token in the vocabulary is mapped to a vector of size vocab_size
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    # This is the forward method, which will calculate logits (predictions) and loss
    def forward(self, index, targets=None):
        # Lookup the embeddings for the given indices (input tokens)
        logits = self.token_embedding_table(index)  # Shape: [B, T, C] (Batch, Time, Vocab size)

        # If targets are not provided, just return the logits (no loss calculation)
        if targets is None:
            loss = None
        else:
            # B = batch size, T = sequence length, C = vocab size (dimensions of the logits tensor)
            B, T, C = logits.shape
            # Reshape targets to be a single long vector (flattening batch and sequence dimensions)
            targets = targets.view(B * T)
            # Reshape logits similarly and calculate cross-entropy loss
            loss = F.cross_entropy(logits.view(B * T, C), targets)

        # Return the logits (for generating predictions) and the loss (for training with gradient descent)
        return logits, loss

    # This method generates new tokens based on a starting index (context)
    def generate(self, index, max_new_tokens):
        # Loop over the number of tokens you want to generate
        for _ in range(max_new_tokens):
            # Get predictions (logits) from the model for the current context
            logits, _ = self.forward(index)  # We don't care about the loss in generation, so it's ignored

            # Focus only on the last time step's logits (for predicting the next token)
            logits = logits[:, -1, :]  # Shape: [B, C] (Last time step, Vocab size)

            # Apply softmax to convert logits into probabilities (summing to 1)
            probs = F.softmax(logits, dim=-1)  # Shape: [B, C] (Batch, Vocab size)

            # Sample a token from the probability distribution (this gives us the next token)
            index_next = torch.multinomial(probs, num_samples=1)  # Shape: [B, 1] (Next token)

            # Append the sampled token to the existing sequence (context), expanding the input
            index = torch.cat((index, index_next), dim=1)  # Shape: [B, T+1] (Add next token to sequence)

        # Return the full generated sequence, including the original context and all new tokens
        return index

model = BigramLanguageModel(vocab_size)

# Step 2: Move the model to the appropriate device (GPU if available, otherwise CPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

# Step 3: Start with an initial context (a single token, represented as index 0 here)
context = torch.zeros((1, 1), dtype=torch.long).to(device)  # Start with a single token in the context

# Step 4: Use the model to generate 50 new tokens based on the context
generated_tokens = decode(model.generate(context, max_new_tokens=500)[0].tolist())

print(generated_tokens)


&pP1:EHrdu—PqNi—roMxn
&VPybAPnM“W’—”cKRlToz?alN.h
lVaDXUJX&&S;-r
-j.“J&”HtW:(M;&C:9zr “tAiGR9mgrT.uKOQ.AftmggdtMZKS
N-0(bJdFz-”dQauTauqY,(9k.rqtLK0Qxnj1—S—er1b ‘,9hm)nMpEFRwyO9l;
)uy&cs1psrV:NLaWy;zRIJWwsP‘&nMqVh
D w&JXFZaV,,’ftrmgHmkMI9rHVBxHZUGI;VY’0tW!O0Bsr!FbAhmqZqrmLFzk.R9,!wsNVK;,SLSRG” WzM?kNzfGZ!!J(zVVqRa”(1iNCY;Yk:mmmLnycsP&CfMv“X E,fPgSj—WOTpe”1YrG!-xp?a”pJXxilO)zCrHCVqrqs9kID WoMSqfiPeej!lTa—BXTk.w-uKgWjOTFZCUlurq”k.ur)oEvHQ—!-:,auh1e-D!Y I!fyC&xt:le9VHgHUrO,,’ErHTv0i-”EzbAuTZm wairHr


In [25]:
eval_iters = 2500

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


step: 0, train loss: 3.0436, val loss: 3.0406
step: 2500, train loss: 2.8336, val loss: 2.8307
step: 5000, train loss: 2.6943, val loss: 2.6960
step: 7500, train loss: 2.5984, val loss: 2.6168
2.760890007019043
